# Test Internal Links

Scan HTML pages to confirm that all internal links are working.

In [1]:
import os
import pandas as pd
from pathlib import Path
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm

In [2]:
here = Path.cwd()
html_root = (here / '../docs/').resolve()

In [3]:
html_files = sorted(list(html_root.rglob('*.html')))

# Add the Mapbox CSVs to test their links as well
html_files += [Path(html_root / '../uploads/to-mapbox-label-points-2012-data.csv').resolve()]
html_files += [Path(html_root / '../uploads/to-mapbox-label-points-2022-data.csv').resolve()]

In [4]:
df = pd.DataFrame()

for h in tqdm(html_files):

    with h.open('r') as f:
        html_text = f.read()

    df_file = pd.DataFrame()
    soup = BeautifulSoup(html_text, features='html.parser')
    links = [link.get('href') for link in soup.find_all('a')]
    
    df_file['destination'] = links
    df_file['source'] = h
    
    df = pd.concat([df, df_file], ignore_index=True)

In [5]:
# Total number of links
len(df)

30791

In [6]:
# Unique link destinations
len(df.destination.unique())
# todo: find orphan HTML pages, those not linked by anything else

4980

In [7]:
df['is_local'] = True
df.loc[df.destination.str.contains('http'), 'is_local'] = False
df_local = df[df.is_local].copy()

In [8]:
df_local['exists'] = df_local.apply(
    lambda x: (Path(x.source).parent / Path(x.destination)).resolve().exists(), axis=1
)

In [9]:
# links_to_check = df[df.is_local].groupby('destination').size()
# links_to_check

In [10]:
df_local['is_broken'] = (df_local.is_local) & ~(df_local.exists)

In [11]:
df_local['source_filename'] = df_local.source.apply(lambda x: Path(x).name)

In [12]:
df_local.is_broken.sum()

0

In [13]:
df_local[df_local.is_broken].source.unique().tolist()

[]

In [14]:
# sorted(list(df_local[df_local.is_broken].destination.unique()))

In [15]:
# df_local[df_local['source_filename'] == '1A.html']

## External Links

In [16]:
# df[~df.is_local].destination.unique()